## 1. Initializing the Dataset

In [3]:
import numpy as np
import pandas as pd 
import torch
from tqdm.notebook import tqdm

### For Google Cloud Platform

In [ ]:
from google.cloud import storage
from io import BytesIO
client = storage.Client()
bucket_name = "your_bucket"
file_name = "your_file_name"
bucket = client.get_bucket(bucket_name)
blob = bucket.get_blob(file_name)
content = blob.download_as_string()
df = pd.read_csv(BytesIO(content),names = ['id','text','category'])
print(df)
df.set_index('id',inplace = True)

                      id                                               text  \
0     611857364396965889  @aandraous @britishmuseum @AndrewsAntonio Merc...   
1     614484565059596288  Dorian Gray with Rainbow Scarf #LoveWins (from...   
2     614746522043973632  @SelectShowcase @Tate_StIves ... Replace with ...   
3     614877582664835073  @Sofabsports thank you for following me back. ...   
4     611932373039644672  @britishmuseum @TudorHistory What a beautiful ...   
...                  ...                                                ...   
3080  613678555935973376  MT @AliHaggett: Looking forward to our public ...   
3081  613294681225621504                    @britishmuseum Upper arm guard?   
3082  615246897670922240           @MrStuchbery @britishmuseum Mesmerising.   
3083  613016084371914753  @NationalGallery The 2nd GENOCIDE against #Bia...   
3084  611566876762640384  @britishmuseum Experience #battlewaterloo from...   

          category  
0           nocode  
1        

### For Normal Computer

In [5]:
df = pd.read_csv('C:/Users/adity/OneDrive/Documents/Work/dataset.csv',names = ['id','text','category'])
print(df)
df.set_index('id',inplace = True)

                      id                                               text  \
0     611857364396965889  @aandraous @britishmuseum @AndrewsAntonio Merc...   
1     614484565059596288  Dorian Gray with Rainbow Scarf #LoveWins (from...   
2     614746522043973632  @SelectShowcase @Tate_StIves ... Replace with ...   
3     614877582664835073  @Sofabsports thank you for following me back. ...   
4     611932373039644672  @britishmuseum @TudorHistory What a beautiful ...   
...                  ...                                                ...   
3080  613678555935973376  MT @AliHaggett: Looking forward to our public ...   
3081  613294681225621504                    @britishmuseum Upper arm guard?   
3082  615246897670922240           @MrStuchbery @britishmuseum Mesmerising.   
3083  613016084371914753  @NationalGallery The 2nd GENOCIDE against #Bia...   
3084  611566876762640384  @britishmuseum Experience #battlewaterloo from...   

          category  
0           nocode  
1        

In [6]:
df.head(10)

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy
614456889863208960,"@britishmuseum say wot, mate?",nocode
614016385442807809,Two workshops on evaluating audience engagemen...,nocode
610916556751642624,"A Forest Road, by Thomas Gainsborough 1750 Oil...",nocode


In [7]:
df.text.iloc[21]

'@nationalgallery #AskTheGallery Why do you pay your brilliant staff so poorly?'

In [8]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [9]:
df = df[df.category.isin(['happy', 'not-relevant', 'angry', 'surprise', 'sad', 'disgust'])]

In [10]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [11]:
possible_labels = df.category.unique()

In [12]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [13]:
label_dict

{'happy': 0,
 'not-relevant': 1,
 'angry': 2,
 'disgust': 3,
 'sad': 4,
 'surprise': 5}

In [14]:
df.category = df['category'].map(label_dict)

In [15]:
df.head(10)

,text,category
id,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,0
614877582664835073,@Sofabsports thank you for following me back. ...,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,0
614499696015503361,Lucky @FitzMuseum_UK! Good luck @MirandaStearn...,0
613601881441570816,Yr 9 art students are off to the @britishmuseu...,0
613696526297210880,@RAMMuseum Please vote for us as @sainsbury #s...,1
610746718641102848,#AskTheGallery Have you got plans to privatise...,1


## 2. Training/Validation Split

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.category.values, 
                                                  test_size=0.15, 
                                                  random_state=42,
                                                  stratify=df.category.values)

In [18]:
df['data_type'] = ['not_set']*df.shape[0]

In [19]:
df.head()

,text,category,data_type
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,0,not_set
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,0,not_set
614877582664835073,@Sofabsports thank you for following me back. ...,0,not_set
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,0,not_set
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,0,not_set


In [20]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [21]:
df.groupby(['category', 'data_type']).count()

text
category data_type      
0        train       966
         val         171
1        train       182
         val          32
2        train        48
         val           9
3        train         5
         val           1
4        train        27
         val           5
5        train        30
         val           5

## 3. Tokenization 

In [24]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [25]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

C:\Users\adity\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adity\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [26]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    max_length=256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].category.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].category.values)

In [27]:
dataset_train = TensorDataset(input_ids_train, 
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val, 
                            attention_masks_val,
                           labels_val)

In [28]:
len(dataset_train)

1258

In [29]:
dataset_val.tensors

(tensor([[  101,  1030,  8223,  ...,     0,     0,     0],
         [  101, 13970, 12269,  ...,     0,     0,     0],
         [  101,  1030,  2358,  ...,     0,     0,     0],
         ...,
         [  101,  2023,  2003,  ...,     0,     0,     0],
         [  101,  1030,  2329,  ...,     0,     0,     0],
         [  101,  2204,  2000,  ...,     0,     0,     0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 2, 4, 0, 1, 0, 1, 0,
         0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 2, 4, 0, 0,
         1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 1, 0, 0, 0, 0, 1, 0, 4, 0, 2, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 2, 1, 1, 0, 3, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0,

## 4. Importing BERT Pre-Trained Model and Creating Data-Loaders

In [30]:
from transformers import BertForSequenceClassification

In [31]:
model = BertForSequenceClassification.from_pretrained(
                                      'bert-base-uncased', 
                                      num_labels = len(label_dict),
                                      output_attentions = False,
                                      output_hidden_states = False
                                     )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [32]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [33]:
batch_size = 4

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

## 5. Setting up Optimizer and Scheduler

In [34]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [35]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

C:\Users\adity\AppData\Roaming\Python\Python39\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [36]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps = len(dataloader_train)*epochs
)

## 6. Setting up our Performance Metrics

In [37]:
import numpy as np
from sklearn.metrics import f1_score

In [38]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

In [39]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

## 7. Setting up our Training Loop

In [40]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [41]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cpu


In [42]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [43]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch), 
                        leave=False, 
                        disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})     

    
    tqdm.write('\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/315 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.795236701884913


  0%|          | 0/7 [00:00<?, ?it/s]

Validation loss: 0.584237494638988
F1 Score (weighted): 0.7639546175365834


...

Validation loss: 0.5275134593248367
F1 Score (weighted): 0.8877840408725088


Epoch 10:   0%|          | 0/315 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.02065978669607642


  0%|          | 0/7 [00:00<?, ?it/s]

Validation loss: 0.5378196835517883
F1 Score (weighted): 0.8877840408725088


## Model Evaluation on Validation

In [44]:
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy:164/171

Class: not-relevant
Accuracy:25/32

Class: angry
Accuracy:4/9

Class: disgust
Accuracy:0/1

Class: sad
Accuracy:2/5

Class: surprise
Accuracy:4/5

